In [1]:
import fastai.vision.all
import numpy as np
import torch
import torchvision
import matplotlib.pyplot as plt
import mpl_toolkits.mplot3d
import time

In [2]:
FUNC1_KIND=3
def Func1(x):
  #NOTE: Switch the function to be learned.
  global FUNC1_KIND
  if FUNC1_KIND==0:  return x
  if FUNC1_KIND==1:  return 0.5*x**2
  if FUNC1_KIND==2:  return 0.1*x**3
  if FUNC1_KIND==3:  return 0.1*x**3-1.0*x

FUNC2_KIND=3
def Func2(x):
  #NOTE: Switch the function to be learned.
  global FUNC2_KIND
  if FUNC2_KIND==0:  return x
  if FUNC2_KIND==1:  return (x[:,0]**2).reshape((-1,1))
  if FUNC2_KIND==2:  return (3.0-x[:,0]).reshape((-1,1))
  if FUNC2_KIND==3:  return (x[:,0]+3.0*np.sin(x[:,0])).reshape((-1,1))
  if FUNC2_KIND==4:  return np.where(x[:,0]**2<1.0, 3.0, 0.0).reshape((-1,1))

def GenerateSample(xmin, xmax, N_sample, noise=1.0e-10):
  data_x1= np.random.uniform(xmin,xmax,size=(N_sample,1))
  data_x2= Func1(data_x1) + np.random.normal(scale=noise,size=(N_sample,1))
  data_y= Func2(data_x2) + np.random.normal(scale=noise,size=(N_sample,1))
  return data_x1,data_x2,data_y

In [3]:
xmin,xmax= -5.0, 5.0
N_sample= 2000
#NOTE: Adjust the sample size and noise level.
data_x1,data_x2,data_y= GenerateSample(xmin, xmax, N_sample, noise=0.1)
#Convert data to torch variables.
data_x1= torch.autograd.Variable(torch.from_numpy(data_x1).float())
data_x2= torch.autograd.Variable(torch.from_numpy(data_x2).float())
data_y= torch.autograd.Variable(torch.from_numpy(data_y).float())

#NOTE: Adjust the batch and epoch sizes.
N_batch= 50

#Make torch dataset and loader.
test_ratio= 0.2
idxes= np.random.RandomState(seed=42).permutation(data_x1.shape[0])
N_train= round(data_x1.shape[0]*(1.-test_ratio))
dset_train= torch.utils.data.TensorDataset(data_x1[idxes[:N_train]], data_x2[idxes[:N_train]], data_y[idxes[:N_train]])
dset_test= torch.utils.data.TensorDataset(data_x1[idxes[N_train:]], data_x2[idxes[N_train:]], data_y[idxes[N_train:]])
# dl_train= torch.utils.data.DataLoader(
#       dataset=dset_train,
#       batch_size=N_batch,
#       shuffle=True,
#       num_workers=2)
# dl_test= torch.utils.data.DataLoader(
#       dataset=dset_train,
#       batch_size=N_batch,
#       shuffle=True,
#       num_workers=2)
#Create fastai dataloaders from torch dataloaders.
dset_train.n_inp=1  #Trick to tell dataloaders that the numer of input is 1.
dset_test.n_inp=1
dls= fastai.data.core.DataLoaders.from_dsets(dset_train, dset_test, bs=N_batch) 
#, tfms=[[None],[None],[None]], n_inp=1, before_batch=lambda x:x)  #NOTE: n_inp

In [4]:
# dls.summary()

In [5]:
print(dls.n_inp)
print(dls.train_ds[:2])
print(dls.after_item, dls.before_batch, dls.after_batch)
# dls.one_batch()
# fastai.data.core.Datasets

1
(tensor([[ 3.4038],
        [-4.5539]]), tensor([[ 0.4462],
        [-5.0740]]), tensor([[ 1.7203],
        [-2.1260]]))
Pipeline:  Pipeline:  Pipeline: 


In [6]:
class TFCN1(torch.nn.Module):
  def __init__(self, p_dropout=0.02):
    super(TFCN1,self).__init__()
    self.net_fc1= torch.nn.Sequential(
          torch.nn.Linear(1, 100),
          torch.nn.LeakyReLU(inplace=True),
          torch.nn.Dropout(p=p_dropout),
          torch.nn.Linear(100, 100),
          torch.nn.LeakyReLU(inplace=True),
          torch.nn.Dropout(p=p_dropout),
          torch.nn.Linear(100, 1),
          )
    self.net_fc2= torch.nn.Sequential(
          torch.nn.Linear(1, 200),
          torch.nn.LeakyReLU(inplace=True),
          torch.nn.Dropout(p=p_dropout),
          torch.nn.Linear(200, 200),
          torch.nn.LeakyReLU(inplace=True),
          torch.nn.Dropout(p=p_dropout),
          torch.nn.Linear(200, 200),
          torch.nn.LeakyReLU(inplace=True),
          torch.nn.Dropout(p=p_dropout),
          torch.nn.Linear(200, 1),
          )
  def forward(self, x):
    x2= self.net_fc1(x)
    y= self.net_fc2(x2)
    return x2,y

In [7]:
net= TFCN1()

In [8]:
#update_method= 'both'
#update_method= 'only_y'
update_method= 'delayed'
i_delay= 20

# def on_loss_begin(last_output, last_target, **kwargs):
#     self.loss2 = loss_func2(last_output, last_target)

# def on_backward_begin(loss, **kwargs):
#     return loss + self.loss2

def loss(output, *target, **kwargs):
    print(f'debug <({type(output)}) {output}> <({type(target)}) {target}>',)
    x2_out, y_out= output
    x2_trg, y_trg= target
    loss_x2= torch.nn.functional.mse_loss(x2_out, x2_trg)
    loss_y= torch.nn.functional.mse_loss(y_out,y_trg)
    return loss_x2 + loss_y

def metric(output, *target, **kwargs):
    x2_out, y_out= output
    x2_trg, y_trg= target
    loss_y= torch.nn.functional.mse_loss(y_out,y_trg)
    return torch.sqrt(loss_y)

fastai.vision.all.MSELossFlat()
fastai.vision.all.rmse
learn= fastai.vision.all.Learner(dls, net,  loss_func=loss, metrics=metric)

## NOTE
When the network returns multiple outputs, using Learner.predict is not good since it outputs only the fast element of the prediction.

```
def predict(self, item, rm_type_tfms=None, with_input=False):
    dl = self.dls.test_dl([item], rm_type_tfms=rm_type_tfms, num_workers=0)
    inp,preds,_,dec_preds = self.get_preds(dl=dl, with_input=True, with_decoded=True)
    ...
    res = dec_targ,dec_preds[0],preds[0]    #<----------HERE!
    if with_input: res = (dec_inp,) + res
    return res
```

In [13]:
x= torch.from_numpy(np.random.uniform(xmin,xmax,size=(1,1))).float()
print(f'x={x}')
# with learn.no_bar(): print(f'  learn.predict(x)={learn.predict(x)}')
with learn.no_bar(): print(f'  learn.get_preds(x)={learn.get_preds(dl=learn.dls.test_dl([x]))[0]}')
net.eval()
with torch.no_grad(): print(f'  net(x)={net(x)}')
print()

x= torch.from_numpy(np.random.uniform(xmin,xmax,size=(3,1))).float()
print(f'x={x}')
# with learn.no_bar(): print(f'  learn.predict(x)={learn.predict((x,torch.zeros(x.shape[0])))[1]}')
# with learn.no_bar(): print(f'  learn.get_preds(x)={learn.get_preds(dl=learn.dls.test_dl((x,torch.zeros(x.shape[0]))))}')
net.eval()
with torch.no_grad(): print(f'  net(x)={net(x)}')

x=tensor([[0.3151]])
  learn.get_preds(x)=(tensor([[-0.0609]]), tensor([[-0.0849]]))
  net(x)=(tensor([[-0.0609]]), tensor([[-0.0849]]))

x=tensor([[ 1.7294],
        [ 2.1766],
        [-1.5137]])
  net(x)=(tensor([[ 0.0647],
        [ 0.1126],
        [-0.1815]]), tensor([[-0.0849],
        [-0.0844],
        [-0.0819]]))


In [14]:
x= torch.from_numpy(np.random.uniform(xmin,xmax,size=(1,1))).float()
print(f'x={x}')
print(f'  learn.predict(x)={learn.predict(x)}')
print(f'  learn.get_preds(x)={learn.get_preds(dl=learn.dls.test_dl([x]))}')  #, with_input=True, with_decoded=True

x=tensor([[4.9796]])


  learn.predict(x)=((tensor([0.2936]), tensor([-0.0821])), tensor([[0.2936]]), tensor([[0.2936]]))


  learn.get_preds(x)=((tensor([[0.2936]]), tensor([[-0.0821]])), None)


In [24]:
x= torch.from_numpy(np.random.uniform(xmin,xmax,size=(3,1))).float()
_dl= learn.dls.test_dl(x,torch.zeros((x.shape[0],1)),torch.zeros((x.shape[0],1)))
# learn.predict([x])[0]
learn.get_preds(dl=_dl)
# learn.dls.n_inp, dls.n_inp

debug <(<class 'tuple'>) (tensor([-0.3740]), tensor([-0.0763]))> <(<class 'tuple'>) (tensor([2.4566]), tensor([-1.7406]))>


IndexError: index 1 is out of bounds for dimension 0 with size 1

In [12]:
learn.summary()

TFCN1 (Input shape: 50)
Layer (type)         Output Shape         Param #    Trainable 
                     50 x 100            
Linear                                    200        True      
LeakyReLU                                                      
Dropout                                                        
Linear                                    10100      True      
LeakyReLU                                                      
Dropout                                                        
____________________________________________________________________________
                     50 x 1              
Linear                                    101        True      
____________________________________________________________________________
                     50 x 200            
Linear                                    400        True      
LeakyReLU                                                      
Dropout                                                 